#Tratamento de Dados

Neste caderno aplicaremos a técnicas de tratamento de dados para os casos em que os dados apresentem valores ausentes e variáveis categóricas.

Materiais de Referência: https://www.kaggle.com/alexisbcook/missing-values

https://www.kaggle.com/alexisbcook/categorical-variables

## Valores Ausentes

No exemplo, trabalharemos com o conjunto de dados __[Melbourne Housing dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)__. Nosso modelo usarEMOS informações como o número de quartos e o tamanho da terra para prever o preço da casa.



---


> Não vamos nos concentrar na etapa de carregamento de dados. Em vez disso, você pode imaginar que já está com os dados de treinamento e validação em X_train, X_valid, y_train e y_valid.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('melb_data.csv')

# Saída do preditor
y = data.Price

# Usaremos apenas preditores numéricos
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide os dados em subconjuntos de treinamento e validação
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,4.0,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,3.0,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,3.0,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [ ]:
# Forma dos dados de treinamento (num_rows, num_columns)
print(X_train.shape)

# Número de valores ausentes em cada coluna de dados de treinamento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Definir função para medir a qualidade de cada abordagem
Definimos uma função *score_dataset()* para comparar diferentes abordagens para lidar com valores ausentes. Esta função relata o **erro absoluto médio (MAE)** de um modelo de floresta aleatório.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 1) Eliminando colunas com valores ausentes
Como estamos trabalhando com conjuntos de **treinamento e validação**, tomamos o cuidado de **eliminar as mesmas colunas nos dois DataFrames**.

In [ ]:
# Obter nomes de colunas com valores ausentes
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Eliminar colunas nos dados de treinamento e validação
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE da Abordagem 1 (Eliminar colunas com valores ausentes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE da Abordagem 1 (Eliminar colunas com valores ausentes):
183550.22137772635


### 2) Imputação
Em seguida, usamos o *SimpleImputer* para substituir os valores ausentes pelo **valor médio em cada coluna**.

Embora seja simples, o preenchimento do valor médio geralmente funciona muito bem (mas isso varia de acordo com o conjunto de dados). Embora os estatísticos tenham experimentado maneiras mais complexas de determinar valores imputados (como imputação de regressão, por exemplo), **as estratégias complexas geralmente não oferecem benefícios adicionais depois que você conecta os resultados a modelos sofisticados de aprendizado de máquina**.

In [ ]:
from sklearn.impute import SimpleImputer

# Imputação
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputação removida de nomes de colunas; colocá-los de volta
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE da Abordagem 2 (Imputação):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE da Abordagem 2 (Imputação):
178166.46269899711


Vemos que a Abordagem 2 tem MAE menor que a Abordagem 1, portanto, a Abordagem 2 teve um desempenho melhor nesse conjunto de dados.

### 3) uma extensão à imputação
Em seguida, imputamos os valores ausentes, além de acompanharmos quais valores foram imputados.

In [ ]:
# Faz uma cópia para evitar a alteração de dados originais (ao imputar)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Faz novas colunas indicando o que será imputado
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputação
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputação removida de nomes de colunas; colocá-los de volta
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Como podemos ver, a abordagem 3 teve um desempenho um pouco pior que a abordagem 2.

### Então, por que a imputação teve um desempenho melhor do que elimiar as colunas?
>
> Os dados de treinamento possuem 10864 linhas e 12 colunas, nas quais três colunas contêm dados ausentes. Para cada coluna, **menos da metade das entradas está ausente**. Assim, a eliminação das colunas remove muitas informações úteis e, portanto, faz sentido que a imputação tenha um desempenho melhor.

## Variáveis Categóricas

Como no tutorial anterior, trabalharemos com o conjunto de dados __[Melbourne Housing dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)__.

Não vamos nos concentrar na etapa de carregamento de dados. Em vez disso, você pode imaginar que já está com os dados de treinamento e validação em X_train, X_valid, y_train e y_valid.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#Exibição dos dados de treinamento
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


Em seguida, obtemos uma lista de todas as **variáveis categóricas nos dados de treinamento**.

Fazemos isso verificando o **tipo de dados** (ou dtype) de cada coluna. O objeto dtype indica que uma coluna tem texto (há outras coisas que teoricamente poderiam ser, mas isso não é importante para nossos propósitos). Para esse conjunto de dados, as colunas com texto indicam variáveis categóricas.

In [ ]:
# Obter lista de variáveis categóricas
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Variáveis Categóricas:")
print(object_cols)

Variáveis Categóricas:
['Type', 'Method', 'Regionname']


### Definir função para medir a qualidade de cada abordagem
Criaremos uma função *score_dataset()* para comparar as três abordagens diferentes para lidar com variáveis categóricas. Esta função relata **o erro absoluto médio (MAE) de um modelo de floresta aleatório**. Em geral, queremos que o MAE seja o mais baixo possível!

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Função para comparar diferentes abordagens
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 1) Descarte de variáveis categóricas
Soltamos as colunas do objeto com o método *select_dtypes()*.

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE da Abordagem 1 (Descarte de variáveis categóricas):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

MAE da Abordagem 1 (Descarte de variáveis categóricas):
175703.48185157913


### 2) Codificação de etiqueta
O *Scikit-learn* possui uma classe *LabelEncoder* que pode ser usada para obter codificações de etiquetas. Repetimos as variáveis categóricas e aplicamos o codificador de rótulo separadamente a cada coluna.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Faz uma cópia para evitar mudanças nos dados originais
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Aplica o codificador de rótulos para cada coluna com dados categóricas
label_encoder = LabelEncoder()
for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

print("MAE da abordagem 2 (Codificação de etiqueta):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE da abordagem 2 (Codificação de etiqueta):
165936.40548390493


Na célula de código acima, para cada coluna, atribuímos aleatoriamente cada valor exclusivo a um número inteiro diferente. Essa é uma abordagem comum que é mais simples do que fornecer rótulos personalizados; no entanto, podemos esperar um aumento adicional no desempenho se fornecermos rótulos mais bem informados para todas as variáveis ordinais.

### 3) Codificação One-hot
Usamos a classe *OneHotEncoder *do *scikit-learn* para obter codificações únicas. Há vários parâmetros que podem ser usados para personalizar seu comportamento:

 - Definimos *handle_unknown = 'ignore'* para evitar erros quando os dados de validação contêm classes que não são representadas nos dados de treinamento;
 - A configuração *sparse = False* garante que as colunas codificadas sejam retornadas como uma matriz numpy (em vez de uma matriz esparsa). <br>
 
Para usar o codificador, fornecemos apenas as colunas categóricas que queremos que sejam codificadas. Por exemplo, para codificar os dados de treinamento, fornecemos *X_train [object_cols]*. (*object_cols* na célula de código abaixo é uma lista dos nomes das colunas com dados categóricos e, portanto, *X_train [object_cols]* contém todos os dados categóricos no conjunto de treinamento.)

In [ ]:
from sklearn.preprocessing import OneHotEncoder


# Aplica a codificador one-hot encoder em cada coluna om dados categóricos
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

print("MAE da abordagem 3 (Codificador One-Hot):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE da abordagem 3 (Codificador One-Hot):
166089.4893009678


### Qual abordagem é a melhor?
Nesse caso, o descarte das colunas categóricas (Abordagem 1) teve pior desempenho, uma vez que teve a maior pontuação no MAE. Quanto às outras duas abordagens, uma vez que as pontuações retornadas do MAE são tão próximas em valor, não parece haver nenhum benefício significativo para uma sobre a outra.

Em geral, a codificação one-hot (Abordagem 3) geralmente apresenta melhor desempenho e a eliminação das colunas categóricas (Abordagem 1) geralmente apresenta pior desempenho, mas varia de acordo com o caso.